In [80]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device used: {device.type}')

torch.cuda.is_available()

Device used: cpu


False

# Hyperparameters for project

In [81]:
train_size = 0.7
test_size = 1 - train_size
num_epochs = 5
batch_size = 4
learning_rate = 0.001

# Data preprocessing


## Import the dataset
Dataset should be imported as a pytorch dataloader for batch optimization

Custom Dataset

## **Data augmentation**

To augment the data before training, we will attempt to use two methods:


1.   Scaling
> Scaling is used because we wish to taken into account the varying structure size of tumors and skulls in the images.

2.   Noise Injection
> Noise injection is used to help assist the model in learning the complex patterns around the tumors and make it more robust to small changes in the data.
> We will experiment with both Gaussian (random), and salt-and-paper (random values to min. or max. values, 0 to 255) noise injection.

All combinations of these will be used to determine their effectiveness, and if they introduce any *bias*, *artifacts*, or *overfitting* both in isolation, or combination.

The order of the data augmentation will be:
1.   No Data Augmentation
2.   Scaling
3.   Noise Injection
4.   Scaling, Noise Injection

In [82]:
from torchvision.transforms import v2

def add_noise_gaussian(tensor, mean = 0, std = 0.05):
    """
    Parameters:
    - tensor: PyTorch tensor data type without noise (input)
    - mean: Mean of the Gaussian distribution
    - std: Standard deviation of the Gaussian distribution

    Returns:
    - tensor + noise: PyTorch tensor data type with noise (output)
    """
    noise = torch.randn(tensor.size()) * std + mean
    return tensor + noise

def add_noise_salt_pepper(tensor, salt_prob = 0.02, pepper_prob = 0.02):
    """
    Parameters:
    - tensor: PyTorch tensor data type without noise (input)
    - salt_prob: Probability that salt noise is added (full white)
    - pepper_prob: Probability that pepper noise is added (full black)

    Returns:
    - tensor + salt_mask = pepper_mask: PyTorch tensor data type with noise (output)
    that ensured to be between 0 and 1
    """

    salt_mask = (torch.rand_like(tensor) < salt_prob).float()
    pepper_mask = (torch.rand_like(tensor) < pepper_prob).float()

    return torch.clamp((tensor + salt_mask - pepper_mask), 0, 1)

# Abitrary values (set to double of base image height*width)
resize_x = 256
resize_y = 256

# Set resize_x and resize_y before using these transforms
# All transforms convert it to a tensor with the dimensions of (Channels, Height, Width)
transforms = {
    'none': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True)
                        ]),
    'scale': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True)
                        ]),
    'noise_gaussian': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Lambda(lambda x: add_noise_gaussian(x))
                        ]),
    'noise_salt_pepper': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Lambda(lambda x: add_noise_salt_pepper(x))
                        ]),
    'all_gaussian':     v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True),
                        v2.Lambda(lambda x: add_noise_gaussian(x))
                        ]),
    'all_salt_pepper':  v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True),
                        v2.Lambda(lambda x: add_noise_salt_pepper(x))
                        ])
}

In [83]:
from torch.utils.data import Dataset
from torchvision.transforms import v2
from PIL import Image
import os

class CustomDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_folders = os.listdir(root_dir)
        self.class_to_idx = {class_folder: i for i, class_folder in enumerate(self.class_folders)}
        self.images = self.make_dataset()

    def make_dataset(self):
        images = []
        for class_folder in self.class_folders:
            class_path = os.path.join(self.root_dir, class_folder)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                images.append((img_path, self.class_to_idx[class_folder]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path)

        random_number = np.random.uniform(0, 1)

        # if self.transform:
        #     image = self.apply_random_transform(image, random_number)
        image = transforms['none'](image)

        return image, label
    
    def apply_random_transform(self, image, random_number):
        # Example: Apply different transformations based on the random number
        if random_number < 0.25:
            return transforms['noise_gaussian'](image)
        elif random_number < 0.5:
            return transforms['noise_salt_pepper'](image)
        else:
            return transforms['none'](image)
    

In [84]:
from torch.utils.data import DataLoader

transform = v2.Compose([
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True)
])

dataset = CustomDataset(root_dir='Dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [85]:
from sklearn.model_selection import train_test_split

train_size = 0.8  # You can adjust the ratio based on your needs
test_size = 1 - train_size
batch_size = 16    # Adjust according to your needs

# Assuming your dataset is a list or any data structure that can be split
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Data normalization

# Architecture of the network

**CNN Model Creation**

In [86]:
class CustomCNN(nn.Module):
    def __init__(self, weights="DEFAULT"):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # MRI images are grayscale, so in_channels=1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)  # Adjust for the flattened conv3 output
        self.fc2 = nn.Linear(256, 4)  # 4 classes in our dataset

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Wrapper for Common NN Parameter Search

In [88]:
model = CustomCNN()

In [89]:
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR

def select_optimizer(optimizer_name, parameters, lr=1e-3, weight_decay=0.):
    if optimizer_name == "sgd":
        return torch.optim.SGD(parameters, lr=lr, weight_decay=weight_decay, momentum=0.9)
    elif optimizer_name == "rmsprop":
        return torch.optim.RMSprop(parameters, lr=lr, weight_decay=weight_decay, alpha=0.99)
    elif optimizer_name == "adam":
        return torch.optim.Adam(parameters, lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")


# Choose optimizer and regularization hyperparameters
optimizer_name = "adam"   # Could be "sgd", "rmsprop", or "adam"
learning_rate = 0.001
weight_decay = 0.0

optimizer = select_optimizer(optimizer_name=optimizer_name, parameters=model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [90]:
def train(model, device, train_loader, optimizer, scheduler, epoch):
    # Set the model in "training" mode, enabling features like dropout and
    # batch normalization that are specific to training.
    model.train()

    # We iterate over `train_loader`, which batches the training data.
    # `enumerate(train_loader)` gives us a counter `batch_idx` and the data in data and target labels in target.
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move our data and labels to the device we are using (CPU or GPU),
        # enabling accelerated computation.
        data, target = data.to(device), target.to(device)

        # Clear old gradients; if not cleared, they would accumulate with subsequent backward passes.
        optimizer.zero_grad()

        output = model(data)  # Forward pass to get predictions.
        loss = F.cross_entropy(output, target)
        loss.backward()       # Backpropagation to compute the gradients.

        # Update the weights of the model based on the gradients calculated during backpropagation.
        optimizer.step()

        if batch_idx % 100 == 0:
            current_lr = scheduler.get_last_lr()[0]  # Access the last learning rate
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} "
                  f"({100. * batch_idx / len(train_loader):.0f}%)]\t"
                  f"Loss: {loss.item():.6f} (LR: {current_lr:.6f})")

    # Update the learning rate according to the specified schedule
    scheduler.step()

In [91]:
def test(model, device, test_loader):
    # Set the model to "evaluation" mode. This is necessary because certain layers
    # like dropout layers behave differently during training than during testing.
    model.eval()

    test_loss = 0
    correct = 0

    # Context manager under which all the operations will have `requires_grad=False`,
    # meaning that PyTorch will not calculate or keep track of gradients.
    # This is used because gradient computation is not needed for evaluation and saves memory and computation.
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # The loss is summed up across all batches.
            # The reduction="sum" parameter ensures that the losses are added together.
            test_loss += F.cross_entropy(output, target, reduction="sum").item()

            # Get the index of the max log-probability
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    # Total loss is divided by the number of items in the dataset to get the average loss.
    test_loss /= len(test_loader.dataset)
    # Accuracy is calculated as the percentage of correct predictions over the total number of predictions.
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n")

**Hyperparameter Gridsearch**

The following code will find the optimal hyperparameters for the mode.

In [92]:
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV


lr = 0.001
epochs = 20
optimizer = torch.optim.Adam
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

net = NeuralNetClassifier(
    module = CustomCNN,
    max_epochs=epochs,
    lr=lr,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    verbose=1
)

params = {
    'lr': [0.001, 0.01, 0.1],
    'max_epochs': list(range(20,55,5)),
    'optimizer': [torch.optim.Adam, torch.optim.SGD, torch.optim.RMSprop],
}

gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=1)
counter = 0
search_batches = 2

for i, data in enumerate(train_loader):
    counter += 1
    image, labels = data
    image = image.to(device)
    labels = labels.to(device)
    outputs = gs.fit(image, labels)
    # GridSearch for `search_batches` number of times.
    if counter == search_batches:
        break
    
print('SEARCH COMPLETE')
print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))
    

Fitting 3 folds for each of 63 candidates, totalling 189 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.3349       0.5000        1.7168  0.1216


c:\Users\benji\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
c:\Users\benji\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


      2        0.7965       0.0000        4.0431  0.1126
      3        0.7239       0.5000        6.3562  0.1073
      4        1.4612       0.5000        5.0112  0.1089
      5        0.6397       0.0000        4.7025  0.1060
      6        0.9228       0.0000        3.7197  0.1141
      7        0.8262       0.0000        2.7979  0.1150
      8        0.7081       0.5000        2.2778  0.1166
      9        0.6912       0.5000        2.0853  0.0985
     10        0.6976       0.5000        2.1051  0.1145
     11        0.6932       0.5000        2.2560  0.1064
     12        0.6861       0.5000        2.4724  0.1178
     13        0.6802       0.5000        2.7031  0.1127
     14        0.6675       0.5000        2.9229  0.1037
     15        0.6466       0.5000        3.1613  0.0945
     16        0.6295       0.5000        3.4579  0.1220
     17        0.6326       0.5000        3.7493  0.1105
     18        0.6212       0.5000        4.0775  0.1089
     19        0.5995       0.5

c:\Users\benji\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


      2        0.8156       0.6667        0.6834  0.1044
      3        0.6600       0.6667        1.3136  0.1068
      4        1.4743       0.3333        0.8304  0.1229
      5        0.7440       0.3333        0.9468  0.1141
      6        0.8420       0.3333        0.6914  0.1107
      7        0.6598       0.6667        0.6356  0.1155
      8        0.6480       0.6667        0.6483  0.1219
      9        0.6782       0.6667        0.6531  0.1251
     10        0.6824       0.6667        0.6536  0.1334
     11        0.6725       0.6667        0.6596  0.1135
     12        0.6633       0.6667        0.6733  0.1034
     13        0.6587       0.6667        0.6805  0.1056
     14        0.6513       0.6667        0.6632  0.1171
     15        0.6306       0.6667        0.6417  0.1151
     16        0.6114       0.6667        0.6349  0.1172
     17        0.6014       0.6667        0.6398  0.1189
     18        0.5789       0.3333        0.6680  0.1116
     19        0.5657       0.6

...

      2       54.6222       0.3333        4.0938  0.0978
      3        4.2363       0.6667        0.8499  0.1021
      4        1.3497       0.3333        1.1789  0.1132
      5        1.2005       0.6667        1.0710  0.0987
      6        1.0674       0.3333        0.7790  0.1003
      7        0.9954       0.6667        1.2018  0.1143
      8        1.3189       0.6667        0.8475  0.0994
      9        0.9246       0.6667        0.8984  0.0979
     10        0.9695       0.3333        0.9779  0.1043
     11        1.0792       0.3333        0.8884  0.1130
     12        1.0066       0.6667        0.7827  0.0998
     13        0.9058       0.6667        0.7747  0.1004
     14        0.9071       0.6667        0.8202  0.0999
     15        0.9369       0.6667        0.8346  0.1148
     16        0.9104       0.6667        0.9904  0.0987
     17        1.0136       0.6667        0.8158  0.1035
     18        0.8994       0.6667        0.7834  0.0980
     19        0.9161       0.6

c:\Users\benji\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


      3        1.2957       0.5000        1.3587  0.0771
      4        1.2575       0.5000        1.3553  0.0802
      5        1.2162       0.5000        1.3540  0.0817
      6        1.1710       0.5000        1.3570  0.0711
      7        1.1212       0.5000        1.3644  0.0756
      8        1.0673       0.5000        1.3813  0.0751
      9        1.0102       0.5000        1.4084  0.0803
     10        0.9528       0.5000        1.4510  0.0679
     11        0.8986       0.5000        1.5081  0.0886
     12        0.8510       0.5000        1.5777  0.0787
     13        0.8121       0.5000        1.6559  0.0769
     14        0.7812       0.5000        1.7397  0.0664
     15        0.7566       0.5000        1.8232  0.0913
     16        0.7377       0.5000        1.9068  0.0758
     17        0.7224       0.5000        1.9857  0.0901
     18        0.7105       0.5000        2.0630  0.0690
     19        0.7007       0.5000        2.1352  0.0856
     20        0.6932       0.5

c:\Users\benji\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


      3        1.3179       0.6667        1.2809  0.0744
      4        1.2852       0.6667        1.2450  0.0795
      5        1.2508       0.6667        1.2058  0.0833
      6        1.2134       0.6667        1.1609  0.0629
      7        1.1708       0.6667        1.1107  0.0636
      8        1.1234       0.6667        1.0557  0.0953
      9        1.0718       0.6667        0.9979  0.0784
     10        1.0173       0.6667        0.9381  0.0705
     11        0.9621       0.6667        0.8813  0.0805


In [ ]:
epochs = 5

# Determine if a GPU with CUDA support is available and use it; otherwise, use the CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using {device} device.\n")

# Begin the training process
for epoch in range(1, epochs + 1):
    # Train the model for one epoch: go through all batches in the training dataset.
    train(model, device, train_loader, optimizer, scheduler, epoch)

    # After each epoch, evaluate the model on the test dataset to monitor its performance.
    test(model, device, test_loader)

Using cpu device.

Train Epoch: 1 [0/5120 (0%)]	Loss: 1.401258 (LR: 0.001000)
Train Epoch: 1 [1600/5120 (31%)]	Loss: 1.096341 (LR: 0.001000)
Train Epoch: 1 [3200/5120 (62%)]	Loss: 0.967987 (LR: 0.001000)
Train Epoch: 1 [4800/5120 (94%)]	Loss: 0.944627 (LR: 0.001000)

Test set: Average loss: 0.9470, Accuracy: 711/1280 (56%)

Train Epoch: 2 [0/5120 (0%)]	Loss: 0.863494 (LR: 0.000700)
Train Epoch: 2 [1600/5120 (31%)]	Loss: 0.915180 (LR: 0.000700)
Train Epoch: 2 [3200/5120 (62%)]	Loss: 0.484237 (LR: 0.000700)
Train Epoch: 2 [4800/5120 (94%)]	Loss: 0.887029 (LR: 0.000700)

Test set: Average loss: 0.8134, Accuracy: 773/1280 (60%)

Train Epoch: 3 [0/5120 (0%)]	Loss: 0.916118 (LR: 0.000490)
Train Epoch: 3 [1600/5120 (31%)]	Loss: 0.987208 (LR: 0.000490)
Train Epoch: 3 [3200/5120 (62%)]	Loss: 0.911620 (LR: 0.000490)
Train Epoch: 3 [4800/5120 (94%)]	Loss: 0.487895 (LR: 0.000490)

Test set: Average loss: 0.7430, Accuracy: 842/1280 (66%)

Train Epoch: 4 [0/5120 (0%)]	Loss: 0.769571 (LR: 0.000343)
T